In [8]:
from chromalab.observer import Observer, Cone
from chromalab.inks import Neugebauer, InkGamut, CellNeugebauer, Pigment
from chromalab.spectra import Spectra, Illuminant

In [1]:
import numpy as np
from time import perf_counter

In [9]:
%load_ext autoreload
%autoreload 2

In [2]:
n = 4

In [3]:
W = []
for i in range(0, 2 ** n):
    binary_str = format(i, f'0{n}b')
    w = tuple(map(lambda x: int(x), binary_str))
    W.append(w)
W = np.array(W)
W_expanded = W[np.newaxis, :, :]  # Shape: 1 x 16 x 4

In [4]:
def time_operation(K):
    X = np.random.rand(K, 4)
    X_expanded = X[:, np.newaxis, :]  # Shape: K x 1 x 4

    start = perf_counter()
    result = X_expanded * W_expanded
    end = perf_counter()

    elapsed_time = end - start
    return elapsed_time

In [7]:
for K in [1, 10, 100, 10e4, 10e5, 10e6,10e7]:
    elapsed_time = time_operation(int(K))
    print(f"Time for K={int(K):e}: {elapsed_time:.6f} seconds. Total of {elapsed_time / K:.3e} ops/s")

Time for K=1.000000e+00: 0.000045 seconds. Total of 4.500e-05 ops/s
Time for K=1.000000e+01: 0.000185 seconds. Total of 1.845e-05 ops/s
Time for K=1.000000e+02: 0.000087 seconds. Total of 8.738e-07 ops/s
Time for K=1.000000e+05: 0.028223 seconds. Total of 2.822e-07 ops/s
Time for K=1.000000e+06: 0.200950 seconds. Total of 2.010e-07 ops/s
Time for K=1.000000e+07: 1.969377 seconds. Total of 1.969e-07 ops/s
Time for K=1.000000e+08: 36.440734 seconds. Total of 3.644e-07 ops/s


batch size of about 100 seems optimal. what do with information..?

## batch mix

In [12]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import numpy.typing as npt
import torch as th

In [13]:
wavelengths1 = np.arange(390, 701, 1)
wavelengths10 = np.arange(400, 701, 10)

In [21]:
cmy_primaries_dict = {}

primary_fns = [
    "000",
    "001",
    "010",
    "100",
    "011",
    "110",
    "101",
    "111",
]

for fn in primary_fns:
    with open(f'../data/nix/PrintColors/{fn}.csv') as csvf:
        spamreader = csv.reader(csvf, delimiter=';')
        for i, row in enumerate(spamreader):
            if i == 4:
                color_data = np.array(row[33:],dtype=float)
                spectra = Spectra(data=color_data, wavelengths=wavelengths10)
                cmy_primaries_dict[fn] = spectra

In [22]:
Neugebauer(cmy_primaries_dict)

In [25]:
neug = Neugebauer(cmy_primaries_dict)

In [64]:
def time_batch_mix(K):
    X = np.random.rand(K,3)
    start = perf_counter()
    for x in X:
        o = neug.mix(x)
    elapsed_1 = perf_counter() - start
    start = perf_counter()
    print(neug.batch_mix(X).shape)
    elapsed_2 = perf_counter() - start
    return elapsed_1, elapsed_2

In [65]:
for K in [1, 10, 100, 10e4]:
    e1, e2 = time_batch_mix(int(K))
    print(f"Time for K={int(K):e}: {e1:.6f} \t {e2:.6f}")

(1, 31)
Time for K=1.000000e+00: 0.000071 	 0.000086
(10, 31)
Time for K=1.000000e+01: 0.000141 	 0.000034
(100, 31)
Time for K=1.000000e+02: 0.001183 	 0.000502
(100000, 31)
Time for K=1.000000e+05: 0.739053 	 0.061277


In [43]:
neug.weights_array.shape

(8, 3)

In [60]:
a = np.random.rand(100, 3, 1)

In [51]:
Y2 = neug.batch_mix(X)

In [52]:
Y1.shape

(100, 31)

In [53]:
Y2.shape

(100, 31)

In [54]:
np.allclose(Y1, Y2)

True

## batch point cloud gen

In [69]:
trichromat = Observer.trichromat()

In [67]:
gamut = InkGamut(neug)

In [80]:
for K in [10, 100, 1e4, 1e5, 1e6]:
    t0 = perf_counter()
    gamut.get_point_cloud(trichromat, stepsize=0.01, batch_size=K, verbose=False)
    print(K, "\t", perf_counter() - t0)

10 	 2.827317374991253
100 	 1.0229483749717474
10000.0 	 1.4325280420016497
100000.0 	 1.2117704160045832
1000000.0 	 1.1843599590938538


In [85]:
for K in [1e4, 1e5, 1e6, 1e7]:
    t0 = perf_counter()
    a, _ = gamut.get_point_cloud(trichromat, stepsize=0.01, batch_size=K, verbose=True)
    print(a.shape)
    print(K, "\t", perf_counter() - t0)

Generating point cloud: 104it [00:01, 60.96it/s]                                                                                                                                              


(1030301, 3)
10000.0 	 1.7456060419790447


Generating point cloud: 11it [00:01,  7.75it/s]                                                                                                                                               


(1030301, 3)
100000.0 	 1.4561074590310454


Generating point cloud: 2it [00:01,  1.79it/s]                                                                                                                                                


(1030301, 3)
1000000.0 	 1.1283836250659078


Generating point cloud: 1it [00:01,  1.05s/it]                                                                                                                                                

(1030301, 3)
10000000.0 	 1.0714638330973685


In [88]:
points, _ = gamut.get_point_cloud(trichromat)

Generating point cloud: 1it [00:00, 356.26it/s]


In [120]:
random_column = np.random.rand(points.shape[0], 1)
points4 = np.hstack((points, 0.001 * random_column))

In [121]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
pca.fit(points4)
np.sqrt(pca.explained_variance_)

array([ 0.21066414,  0.11139065,  0.02356711,  0.00028971])

In [ ]:
trichromat